# 概要

機械学習において重要トピックの一つであるハイパーパラメータのチューニング方法を実装してみる。

今回california_housingをつかって、ランダムサーチによる探索を試す。

# ランダムサーチ：sklearn.datasets.fetch_california_housing

## パッケージインポート

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense  # layerクラスを直接インポートして使用出来る

##### これうまくいくはずなんだけど学習できてない #####
from tensorflow.keras.losses import MeanSquaredError  # 損失関数クラスを直接インポートして使用出来る
from tensorflow.keras.optimizers import SGD  # オプティマイザクラスを直接インポートして使用出来る
##################################################

import numpy as np
import pandas as pd

print(f'tensorflow ver.{tf.__version__}')
print(f'keras ver.{keras.__version__}')

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

tensorflow ver.2.8.0
keras ver.2.8.0


##  データロードと前処理

### データロード

In [2]:
housing = fetch_california_housing()
x_train_full, x_test, y_train_full, y_test = train_test_split(housing.data, housing.target)


### validation分割

In [3]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full)

# データサイズを確認
print(f'x_train.shape : {x_train.shape}')
print(f'y_train.shape : {y_train.shape}')
print(f'x_valid.shape : {x_valid.shape}')
print(f'y_valid.shape : {y_valid.shape}')

x_train.shape : (11610, 8)
y_train.shape : (11610,)
x_valid.shape : (3870, 8)
y_valid.shape : (3870,)


trainデータを確認

In [4]:
pd_x_train = pd.DataFrame(x_train, columns=housing.feature_names)
display(pd_x_train)
pd_x_train.info()
pd_x_train.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,3.6731,35.0,5.350254,1.027919,1243.0,3.154822,33.92,-118.03
1,2.3774,26.0,4.867804,1.078891,634.0,1.351812,33.60,-117.70
2,3.0347,45.0,4.537500,0.912500,568.0,3.550000,34.07,-118.09
3,1.2056,23.0,3.398844,0.907514,716.0,4.138728,34.00,-118.24
4,2.3507,12.0,5.531073,1.042373,875.0,2.471751,39.82,-121.59
...,...,...,...,...,...,...,...,...
11605,3.1000,29.0,7.542373,1.591525,1328.0,2.250847,38.44,-122.98
11606,3.0472,26.0,3.575453,1.116700,1966.0,3.955734,37.47,-122.21
11607,2.6477,40.0,6.299465,1.304813,586.0,3.133690,36.32,-119.70
11608,3.8257,13.0,5.584027,0.980033,1458.0,2.425957,36.97,-120.08


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11610 entries, 0 to 11609
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MedInc      11610 non-null  float64
 1   HouseAge    11610 non-null  float64
 2   AveRooms    11610 non-null  float64
 3   AveBedrms   11610 non-null  float64
 4   Population  11610 non-null  float64
 5   AveOccup    11610 non-null  float64
 6   Latitude    11610 non-null  float64
 7   Longitude   11610 non-null  float64
dtypes: float64(8)
memory usage: 725.8 KB


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000
mean,3.865153,28.678553,5.416247,1.095383,1419.184927,3.070159,35.622855,-119.562420
std,1.889347,12.631209,2.351461,0.469608,1120.247319,11.754017,2.130761,1.998148
min,0.499900,1.000000,0.888889,0.333333,6.000000,0.692308,32.540000,-124.300000
25%,2.559600,18.000000,4.437482,1.004978,787.000000,2.435037,33.940000,-121.790000
50%,3.536950,29.000000,5.233217,1.047831,1162.000000,2.820312,34.250000,-118.480000
75%,4.740925,37.000000,6.058770,1.098851,1721.750000,3.285013,37.710000,-118.010000
max,15.000100,52.000000,132.533333,34.066667,35682.000000,1243.333333,41.950000,-114.310000


### 前処理

#### スケーリング

### [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)
データの標準化を行う。

代表的なメソッドは以下：

|メソッド|説明|
|---|---|
|fit()|標準化するための平均と分散を計算する。|
|trasform()|（事前に計算した平均と分散を使用して）標準化を行う。|
|fit_transform()|平均と分散を計算し、標準化を行う。|

In [5]:
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_valid = scaler.transform(x_valid)  # x_trainの平均・分散を使用する（のはなぜ？）
x_test = scaler.transform(x_test)  # x_trainの平均・分散を使用する（のはなぜ？）

## モデル作成

複数のモデルを比較しやすくするようにモデルを作成する関数を定義しておく。

この関数では引数で渡した層の数、各層のニューロン数、学習率でSequentialモデル作成→SGDオプティマイザでコンパイルまで行い、モデルを返す。

In [6]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[x_train.shape[1]]):
    # Sequentialモデル生成
    model = keras.models.Sequential()
    # InputLayer追加
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    # hiddenLayer追加
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    # OutputLayer追加
    model.add(keras.layers.Dense(1))

    # optimizer生成
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)

    # compile
    model.compile(loss='mse', optimizer=optimizer)

    return model

# build_modelを使ってKerasRegressorを作る
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

/var/folders/bb/4zdkwm1s52jb7g15whtp3m700000gn/T/ipykernel_68149/3661465457.py:21: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


KerasRegressorオブジェクトはbuild_modelを使って構築されるモデルに薄いラップをかぶせたものである。<br>
（このラップによって？）scikit-learnの回帰モデルを同じようなユーザインタフェースを使用することが出来る。

## 学習と評価

### コールバックによる学習中のチェックポイント保存

今回早期打ち切り設定を入れるためにEarlyStopping関数を使用する。

また、コールバック関数は自作したものを使うことが出来る。<br>
例として学習中の訓練データのlossとvalidationデータのlossの比率を表示する関数を作成する。（過学習を検知すること想定）

In [7]:
# 早期打ち切りのコールバック関数
# 学習打ち切り時に性能が最高だった時の重みを復元するので最良モデルの保存と復元は不要
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)  # patienceで指定したエポック数学習が進まなかったときに学習を打ち切る

In [8]:
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))

print_valid_train_ration_cb = PrintValTrainRatioCallback()

### TensorBoardを使った可視化

In [9]:
import os

# ログ出力のルートディレクトリ
root_dir = os.path.join(os.curdir, 'my_logs')

# ログディレクトリ名を生成する関数
def get_run_logdir():
    import time
    run_id = time.strftime('run_%Y_%m_%d-%H_%M_%S')
    return os.path.join(root_dir, run_id)

run_logdir = get_run_logdir()

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

### 学習

scikit-learnのようにfitメソッドで学習する。<br>
ただし引数はbuild_modelの土台になっているkerasモデルに渡される。<br>
その例として以下ではcallbackを渡している。

In [10]:
# scikit-learnのようにfitメソッドで学習.
keras_reg.fit(x_train, y_train, epochs=100, validation_data=(x_valid, y_valid), callbacks=[early_stopping_cb,print_valid_train_ration_cb,tensorboard_cb])

2022-07-10 21:46:09.033474: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
330/363 [==========================>...] - ETA: 0s - loss: 1.2934
val/train: 0.58
363/363 [==============================] - 1s 2ms/step - loss: 1.2374 - val_loss: 0.7170
Epoch 2/100
352/363 [============================>.] - ETA: 0s - loss: 0.7933
val/train: 1.34
363/363 [==============================] - 0s 1ms/step - loss: 0.7912 - val_loss: 1.0583
Epoch 3/100
298/363 [=======================>......] - ETA: 0s - loss: 1.6960
val/train: 1.17
363/363 [==============================] - 0s 999us/step - loss: 1.4938 - val_loss: 1.7483
Epoch 4/100
357/363 [============================>.] - ETA: 0s - loss: 2.4826
val/train: 0.24
363/363 [==============================] - 0s 915us/step - loss: 2.4536 - val_loss: 0.5881
Epoch 5/100
329/363 [==========================>...] - ETA: 0s - loss: 0.5666
val/train: 0.96
363/363 [==============================] - 0s 817us/step - loss: 0.5589 - val_loss: 0.5345
Epoch 6/100
322/363 [=========================>....] - ETA: 0s - loss: 0.5214
v

In [11]:
%load_ext tensorboard

%tensorboard --logdir ./my_logs

Reusing TensorBoard on port 6006 (pid 59349), started 0:43:02 ago. (Use '!kill 59349' to kill it.)

### 評価

In [12]:
# scikit-learnのようにscoreメソッドで評価(scoreは高いほぼ良い)
mse_test = keras_reg.score(x_test, y_test)
mse_test

162/162 [==============================] - 0s 615us/step - loss: 0.3400


-0.34004828333854675

## 学習済みモデルを使った予測

In [13]:
# サンプル用にデータサイズを限定
x_new = x_test[:3]

# scikit-learnのようにpredictメソッドで予測
y_pred = keras_reg.predict(x_new)
print(f'predict : {y_pred.reshape(-1)}')
print(f'correct : {y_test[:3]}')

predict : [1.5267634 2.6027513 1.0641413]
correct : [1.89  2.523 2.7  ]


## ランダムサーチによるパラメータ探索

ここまででパラメータ探索をするモデル側の準備が出来たので、実際にランダムサーチを使って探索を行う。

## モデルの保存と復元

In [14]:
from scipy.stats import reciprocal  # 逆分布（対数一様分布）
from sklearn.model_selection import RandomizedSearchCV  # ランダムサーチ

param_distribs = {
    'n_hidden' : [0, 1, 2, 3],
    'n_neurons' : np.arange(1, 100),
    'learning_rate' : reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(estimator=keras_reg  # パラメータ探索を行うモデル
                                    , param_distributions=param_distribs  # パラメータ探索を試すパラメータの分布またはリストの辞書
                                    , n_iter=10  # 試行回数
                                    , cv=3  # 交差検証の分割数
                                    )
rnd_search_cv.fit(x_train, y_train, epochs=100, validation_data=(x_valid, y_valid), callbacks=[early_stopping_cb,print_valid_train_ration_cb,tensorboard_cb])

Epoch 1/100
238/242 [============================>.] - ETA: 0s - loss: 5.5557
val/train: 0.80
242/242 [==============================] - 1s 1ms/step - loss: 5.5279 - val_loss: 4.4405
Epoch 2/100
168/242 [===================>..........] - ETA: 0s - loss: 4.0065
val/train: 0.82
242/242 [==============================] - 0s 895us/step - loss: 3.9860 - val_loss: 3.2531
Epoch 3/100
164/242 [===================>..........] - ETA: 0s - loss: 3.1587
val/train: 0.83
242/242 [==============================] - 0s 902us/step - loss: 2.9645 - val_loss: 2.4587
Epoch 4/100
168/242 [===================>..........] - ETA: 0s - loss: 2.4043
val/train: 0.84
242/242 [==============================] - 0s 898us/step - loss: 2.2773 - val_loss: 1.9213
Epoch 5/100
170/242 [====================>.........] - ETA: 0s - loss: 1.8935
val/train: 0.86
242/242 [==============================] - 0s 898us/step - loss: 1.8099 - val_loss: 1.5543
Epoch 6/100
236/242 [============================>.] - ETA: 0s - loss: 1.4867

KeyboardInterrupt: 

fitメソッドに渡した引数はその下のkerasモデルにリレーされている。<br>
また、RandomizedSearchCVは交差検証を使用するため学習精度を評価するためにx_valid, y_validは使わない。これらは早期打ち切りのために使用される。

次のように最良のパラメータ、スコア、モデルを取得できる。

In [ ]:
# 最良モデルのインデックス
print(rnd_search_cv.best_index_)

# 最良モデルのパラメータ
print(rnd_search_cv.best_params_)

# 最良モデルのスコア
print(rnd_search_cv.best_score_)

# 最良モデル
model = rnd_search_cv.best_estimator_.model
print(model.evaluate(x_test, y_test))

# 補足

scipy.statsは確率分布、統計量、仮説検定などの統計に関するモジュールを収録している. <br>
reciprocal は逆分布（対数一様分布）<br>
確率密度関数は
$$ f(x;a,b)={\frac {1}{x[\log _{e}(b)-\log _{e}(a)]}}\quad {\text{ for }}a\leq x\leq b{\text{ and }}a>0. $$
パラメータの説明をすると、$a$は下限, $b$は上限を与える.（←本当かな？）

In [ ]:
from scipy.stats import reciprocal  # 逆分布（対数一様分布）
from matplotlib import pyplot as plt

x = np.linspace(start=0, stop=11, num=10000) 
dist = reciprocal(1, 10)

fig, ax = plt.subplots(1, 1)
ax.plot(x, dist.pdf(x), 'k-', lw=2, label='frozen pdf')
ax.legend(loc='best', frameon=False)
plt.show()